Ссылка на Colab: https://colab.research.google.com/drive/12ZqueyraeLcg8sCLbVvE7all6b8cXBhz?usp=sharing

In [71]:
# Загрузка датасетов из облака google
import gdown

# Функции операционной системы
import os

# Модуль pandas для обработки табличных данных
import pandas as pd

# работа с датами
from datetime import datetime as dt

In [73]:
# Загрузка датасета c Гугл Диска
gdown.download('https://drive.google.com/uc?id=1AjntkjOw1NPk0J-DzEl98W1Ex16v7Xrq', None, quiet=True)

'dosing_data.zip'

In [74]:
# Распаковка архива в папку BashCardio
!unzip -qo dosing_data.zip -d BashCardio

# Просмотр содержимого папки (добавил в архив общий распаршенный датасет для сравнения данных)
!ls BashCardio

'Свод и чистка 8 больниц.xlsx'	 ЦАТ_NEFT.xlsx	 ЦАТ_общий_parsed.xlsx
 ЦАТ_BUZD.xlsx			 ЦАТ_OKT.xlsx	'Чишминская ЦРБ.xlsx'
 ЦАТ_GKB5.xlsx			 ЦАТ_SIB.xlsx
 ЦАТ_KMRT.xlsx			 ЦАТ_STR.xlsx


# Приведение датасетов к удобному виду для анализа

In [4]:
# называем колонки просто цифрами (не придумал ничего оригинальнее)
COLUMN_NAMES = list(range(11))

In [78]:
def apply_replacements(s, list_of_replacements):
  for replacement in list_of_replacements:
     s = s.replace(replacement[0], replacement[1])
  return s

def transform_gender(gender):
# преобразование пола: в результате 0 - женский пол, 1 - мужской
  gender_dict = {'мужской': 1, 'муж': 1, 'М': 1, 'м': 1, 'женский': 0, 'жен': 0, 'ж': 0, 'Ж': 0}

  return gender_dict[gender]

def transform_num(num):
# преобразование чисел (МНО) - приводим все к единому формату, переводим во float
  if type(num) is str:
    list_of_replacements = [('.,', '.'), (',,', '.'), (' , ', '.'), (' ,', '.'), (' .', '.'),
                            (', ', '.'),  (',', '.'), ('27.7время мно нет', '27.7')]

    num = apply_replacements(num, list_of_replacements)

  return float(num)

def transform_date(date):
# преобразование дат - приводим все к единому формату, переводим в datetime.datetime
  if date == '31.11.2022':
    date = '31.12.2022'

  try:
    if date[-1] == '.':
      date = date[:-1]
  except(IndexError):
    print('Ошибка индекса в дате', date=='')

  list_of_replacements = [('01.01.1971Хроническая  ревматическая болезнь сердца. Аортальный стеноз+недостаточность 2-3ст', '01.01.1971'),
                          ('021.11.2022', '21.11.2022'),
                          ('02.023.1949', '02.02.1949'),
                          ('26.12.212', '26.12.2022'), ('06.10.20212', '06.10.2022'),
                          ('ё', '04.01.2023'), ('17.01.23', '17.01.2023'), ('16.11.22.','16.11.2022'),
                          ('г.р.',''), ('г.р',''), ('г.',''), ('г',''), (',','.'),
                          ('.22','.2022'),
                          ('30.9.2022','30.09.2022'), ('29.062022','29.06.2022'),
                          ('14.112022','14.11.2022'), ('18.01.23', '18.01.2023'),
                          ('..', '.')]

  date = apply_replacements(date, list_of_replacements)

  return dt.strptime(date.strip(), '%d.%m.%Y')

def transform_age(date):
  if 'лет' in date or 'год' in date:
    list_of_replacements = [('года', ''), ('год', ''), ('лет', '')]

    return int(apply_replacements(date, list_of_replacements))

  else:
    birthdate = transform_date(date)

    return 2023 - birthdate.year

def load_data(fname, skiprows, col_shift=0):
# загрузка данных в DataFrame, параметры: fname - имя файла Excel с данными,
# skiprows - сколько строк пропускаем, col_shift - в одном из файлов необходимо
# сдвинуть номера колонок, здесь указываем, на какое число,
# возвращает: DataFrame с загруженными данными
  df = pd.read_excel(fname, names=COLUMN_NAMES, skiprows=skiprows)

  result = []
  rows = df.iterrows()
  num = 1
  for row in rows:

    # имя
    if type(row[1][1+col_shift]) is str:
      name = row[1][1+col_shift].strip()

    # пол
    if type(row[1][2+col_shift]) is str:
      try:
        gender = transform_gender(row[1][2+col_shift].strip())
      except(KeyError):
        print(fname, 'Error in gender!', row[1][2+col_shift])

    # дата рождения
    if type(row[1][3+col_shift]) is str and row[1][3+col_shift].strip() != '':
      try:
        age = transform_age(row[1][3+col_shift].strip())
      except(ValueError):
        print(fname, 'Error in birthdate!', row[1][3+col_shift])
    elif type(row[1][3+col_shift]) is dt:
      age = 2023 - row[1][3+col_shift].year

    # препарат
    if type(row[1][6+col_shift]) is str:
      medication = row[1][6+col_shift].strip()
    if type(row[1][8+col_shift]) is str:
      try:
        date = transform_date(row[1][8+col_shift])
      except(ValueError):
        print(fname, 'Error in date!', row[1][8+col_shift])
    else:
      date = row[1][8+col_shift]

    # доза
    try:
      value = transform_num(row[1][9+col_shift])
    except(ValueError, TypeError):
      print(fname, 'Error in value!', row[1][9+col_shift])
      value = float('nan')

    # некоторых пациентов обрабатываем вручную
    if name == 'Валеева Альфия Гафуровна':
      result.append([name, gender, age, dt.strptime('24.11.2022', '%d.%m.%Y'), 2.69])
      result.append([name, gender, age, dt.strptime('16.01.2023', '%d.%m.%Y'), 2.34])
    elif name == 'Александрова Валентина Михайловна' and row[1][8+col_shift] == '25.03.202':
      result.append([name, gender, age, dt.strptime('25.03.2022', '%d.%m.%Y'), 1.24])
    elif name == 'Акчурин Зинфар Зуфарович' and row[1][8+col_shift] == '18.10.202':
      result.append([name, gender, age, dt.strptime('18.10.2022', '%d.%m.%Y'), 1.98])
    elif name == 'Ямщикова Анна Михайловна' and row[1][8+col_shift] == '30.11.2022-2,36':
      result.append([name, gender, age, dt.strptime('30.11.2022', '%d.%m.%Y'), 2.36])
    elif name == 'Лобачева Надежда Романовна' and type(date) is not dt:
      result.append([name, gender, age, dt.strptime('01.11.2022', '%d.%m.%Y'), 2.94])

    # отсекаем только пациентов, принимающих варфарин
    elif 'варфарин' in medication.lower():
      result.append([name, gender, age, date, value])


  result = pd.DataFrame(result).dropna()
  counts = result[0].value_counts()
  result[9] = list(counts[result[0]])

  # отсекаем тех пациентов, которые встречаются более 1 раза
  return result[result[9] > 1][[0,1,2,3,4]]

# Проверка датасетов

Загружаем датасеты из исходных файлов и сводим все в один DataFrame. Выполняем сортировку сначала по имени пациента, потом по дате анализа (это важно, потому что определяет динамику МНО, в оригинальном датасете этого не было сделано, судя по всему).

In [81]:
contents = [('/content/BashCardio/ЦАТ_BUZD.xlsx', 3, 0), ('/content/BashCardio/ЦАТ_GKB5.xlsx', 4, 0), ('/content/BashCardio/ЦАТ_KMRT.xlsx', 4, 0), ('/content/BashCardio/ЦАТ_NEFT.xlsx', 4, 0), ('/content/BashCardio/ЦАТ_OKT.xlsx', 4, 0), ('/content/BashCardio/ЦАТ_SIB.xlsx', 4, 0), ('/content/BashCardio/ЦАТ_STR.xlsx', 5, -1), ('/content/BashCardio/Чишминская ЦРБ.xlsx', 4, 0)]
dfs = []

for elem in contents:
  print(elem[0])
  df1 = load_data(elem[0], elem[1], elem[2])
  dfs.append(df1)

res = pd.concat(dfs)
res.sort_values(by=[0,1], inplace=True)

res.rename(columns = {0: 'patient', 3: 'AnalisDate'}, inplace = True )

res.reset_index(drop=True)

/content/BashCardio/ЦАТ_BUZD.xlsx
/content/BashCardio/ЦАТ_BUZD.xlsx Error in date! 20.05.2022.г
/content/BashCardio/ЦАТ_GKB5.xlsx
/content/BashCardio/ЦАТ_KMRT.xlsx
/content/BashCardio/ЦАТ_KMRT.xlsx Error in date! 24.11.2022 г ., 16.01.2023г
/content/BashCardio/ЦАТ_KMRT.xlsx Error in value! 2,69    ., 2,34
/content/BashCardio/ЦАТ_NEFT.xlsx
/content/BashCardio/ЦАТ_OKT.xlsx
/content/BashCardio/ЦАТ_SIB.xlsx
/content/BashCardio/ЦАТ_STR.xlsx
/content/BashCardio/ЦАТ_STR.xlsx Error in date! 25.03.202
/content/BashCardio/ЦАТ_STR.xlsx Error in date! 18.10.202
/content/BashCardio/ЦАТ_STR.xlsx Error in value! 1944-03-01 00:00:00
/content/BashCardio/ЦАТ_STR.xlsx Error in date! 30.11.2022-2,36
/content/BashCardio/ЦАТ_STR.xlsx Error in value! 2023-08-01 00:00:00
/content/BashCardio/Чишминская ЦРБ.xlsx


,patient,1,2,AnalisDate,4
0,АБРАМОВ ТАТЬЯНА,0,76,2022-12-11,1.50
1,АБРАМОВ ТАТЬЯНА,0,76,2023-01-24,1.80
2,АГЕЕВ АНУВАР ГАБДУЛЛОВИЧ,1,76,2023-01-13,2.30
3,АГЕЕВ АНУВАР ГАБДУЛЛОВИЧ,1,76,2022-12-26,1.89
4,АГЕЕВ АНУВАР ГАБДУЛЛОВИЧ,1,76,2022-09-15,2.67
...,...,...,...,...,...
2565,мирзаянов Альфред Амирьянович,1,67,2023-01-13,3.00
2566,мирзаянов Альфред Амирьянович,1,67,2022-12-09,4.36
2567,тимофеев николай павлович,1,76,2022-12-21,2.00
2568,тимофеев николай павлович,1,76,2023-01-09,2.30


Для удобства переведем все в словарь, ключи - имена пациентов (в нижнем регистре, чтобы удобнее сравнивать), значения - списки МНО пациента, упорядоченные по дате измерения.

In [ ]:
res_dict = {}
name_index = 0
inr_index = 4

res_rows = res.iterrows()
for row in res_rows:
  name = row[1][name_index].lower()
  if name in res_dict:
    res_dict[name].append(row[1][inr_index])
  else:
    res_dict[name] = [row[1][inr_index]]

res_dict, len(res_dict)

({'абрамов татьяна': [1.5, 1.8],
  'агеев анувар габдуллович': [2.67, 1.89, 2.3],
  'ананьев владимир алексеевич': [2.4, 5.8, 2.8],
  'абдрахманов гаряй анварович': [3.41, 1.89, 1.59],
  'абдулганиева насима мазгаровна': [1.9, 2.9, 3.0],
  'абдулгафаров рафкат янузакович': [1.04, 1.67, 1.19],
  'абдулина миннигуль тажитдиновна': [1.53, 3.73, 3.17],
  'абдуллина надежда кирилловна': [2.0, 2.0],
  'абдульманов ильсур минигалеевич': [2.13, 2.11, 2.11, 2.89],
  'абзалов дамир мукадаевич': [2.9, 2.9, 2.6],
  'абзалова нафиса ахметсафиновна': [2.0, 2.0],
  'абросимова роза суфияновна': [1.0, 1.3],
  'абсалямова зугра г': [1.0, 1.0, 1.55],
  'абузарова гузяль шамилевна': [2.88, 3.12, 3.46],
  'абусев фаил рахимуллиевич': [2.29, 1.65, 1.97],
  'аверьянова ирина рафиковна': [1.7, 2.19, 2.2],
  'агафонова татьяна викторовна': [2.56, 2.34, 2.6],
  'агишев тагир фаритович': [2.3, 2.0],
  'аглиуллин филорит бодриевич': [2.5, 2.4, 2.1],
  'аглямова разида инсафовна': [1.36, 1.02, 1.01],
  'адельгаре

Получилось 865 пациентов. Теперь загрузим оригинальный датасет.

In [ ]:
df = pd.read_excel('/content/BashCardio/ЦАТ_общий_parsed.xlsx', sheet_name = [0,1])
df

{0:                     institution               id  gender   age   dose  \
 0     ГБУЗ РБ Толбазинская ЦРБ   140361892057136       0  57.0  7.500   
 1     ГБУЗ РБ Толбазинская ЦРБ   140361892057136       0  57.0  7.500   
 2     ГБУЗ РБ Толбазинская ЦРБ   140361892057264       1  50.0  5.000   
 3     ГБУЗ РБ Толбазинская ЦРБ   140361892057392       0  53.0  6.250   
 4     ГБУЗ РБ Толбазинская ЦРБ   140361892057392       0  53.0  6.250   
 ...                         ...              ...     ...   ...    ...   
 1463     ГБУЗ РБ Янаульская ЦРБ  139766010360000       1  68.0  3.750   
 1464     ГБУЗ РБ Янаульская ЦРБ  139766010360144       1  35.0  3.750   
 1465     ГБУЗ РБ Янаульская ЦРБ  139766010360144       1  35.0  3.750   
 1466     ГБУЗ РБ Янаульская ЦРБ  139766010360288       0  64.0  2.500   
 1467     ГБУЗ РБ Янаульская ЦРБ  139766010360288       0  64.0  1.875   
 
      date_analyse  prev_INR(MHO)  atrial_fibrillation  mitral_valve  \
 0      2022-11-11           3.11  

Соединим со вторым листом, чтобы получить ФИО пациентов (а не только id).

In [ ]:
persons_final = df[0][['id', 'prev_INR(MHO)', 'INR(MHO)', 'institution']].merge(df[1][['id','FIO']], how='left', on='id')
persons_final

,id,prev_INR(MHO),INR(MHO),institution,FIO
0,140361892057136,3.11,3.32,ГБУЗ РБ Толбазинская ЦРБ,Гайнуллин ильшат Раисович
1,140361892057136,3.32,2.01,ГБУЗ РБ Толбазинская ЦРБ,Гайнуллин ильшат Раисович
2,140361892057264,2.01,2.75,ГБУЗ РБ Толбазинская ЦРБ,Алпарова Гульнара Раиловна
3,140361892057392,1.42,2.56,ГБУЗ РБ Толбазинская ЦРБ,Ашмарин Андрей Петрович
4,140361892057392,2.56,2.80,ГБУЗ РБ Толбазинская ЦРБ,Ашмарин Андрей Петрович
...,...,...,...,...,...
1463,139766010360000,1.34,2.30,ГБУЗ РБ Янаульская ЦРБ,Петрова Земфира Вениаминовна
1464,139766010360144,1.50,1.04,ГБУЗ РБ Янаульская ЦРБ,Кутлиахметова Анна Валериевна
1465,139766010360144,1.04,1.03,ГБУЗ РБ Янаульская ЦРБ,Кутлиахметова Анна Валериевна
1466,139766010360288,2.80,3.19,ГБУЗ РБ Янаульская ЦРБ,Шайгадамов Каримьян Шаймухаметович


Проделаем те же действия по переводу в словарь.

In [ ]:
orig_dict = {}

orig_rows = persons_final.iterrows()
for row in orig_rows:
  name = row[1]['FIO'].lower().replace('\xa0', '').strip()
  if name in orig_dict:
    orig_dict[name].append(row[1]['INR(MHO)'])
  else:
    orig_dict[name] = [row[1]['prev_INR(MHO)'], row[1]['INR(MHO)']]

orig_dict, len(orig_dict)

({'гайнуллин ильшат раисович': [3.11, 3.32, 2.01],
  'алпарова гульнара раиловна': [2.01, 2.75],
  'ашмарин андрей петрович': [1.42, 2.56, 2.8],
  'кутлияров ринат рашитович': [2.16, 2.1],
  'рамаданова раиса николаевна': [2.6, 2.1],
  'галеев фанур гизитдинович': [1.5, 3.2],
  'забелин иван петрович': [2.8, 1.98],
  'воробьева ольга васильевна': [1.41, 1.99, 3.16],
  'горина аниса латыповна': [2.18, 2.29, 4.09],
  'логвин сергей лукьянович': [1.58, 1.81, 2.51],
  'волкова надежда петровна': [1.23, 2.11, 2.47],
  'акчурин зинфар зуфарович': [2.7, 1.98, 1.73],
  'аглямова разида инсафовна': [1.02, 1.36, 1.01],
  'назаров рафис шарифьянович': [1.0, 1.95, 1.08],
  'новиков а. в.': [1.63, 2.58, 2.7],
  'уманцева е. с.': [2.02, 2.3, 2.58],
  'битунов виктор петрович': [1.18, 1.18, 1.18],
  'зайнетдинович валеев радик': [1.72, 1.09, 1.2],
  'андреев анатолий иванович': [2.15, 2.72, 2.48],
  'халиков руслан филусович': [2.08, 2.1],
  'сафиуллина минигайша зайнетдиновна': [3.07, 3.28, 2.48],
 

Получилось 767 человек, то есть на 98 меньше! Теперь сравним порядок значений МНО.

In [ ]:
count = 0
for person in res_dict:
  if person in orig_dict:
    if res_dict[person] == orig_dict[person]:
      continue
    else:
      count += 1
      print('В датасете нарушена последовательность измерений для', person, '- в оригинале было', res_dict[person], 'стало', orig_dict[person])
  else:
      print('Не найден ключ', person, 'в датасете')

print ('Нарушена последовательность для', count, 'пациентов')

Не найден ключ абдулгафаров рафкат янузакович в датасете
Не найден ключ аверьянова ирина рафиковна в датасете
В датасете нарушена последовательность измерений для аглямова разида инсафовна - в оригинале было [1.36, 1.02, 1.01] стало [1.02, 1.36, 1.01]
В датасете нарушена последовательность измерений для адельгареева рида курбангалиевна - в оригинале было [2.41, 4.96, 1.98] стало [1.98, 4.96, 2.41]
В датасете нарушена последовательность измерений для аитова разина тагировна - в оригинале было [1.56, 1.57, 2.21] стало [2.21, 1.57, 1.56]
В датасете нарушена последовательность измерений для айдарова лузия галимьяновна - в оригинале было [1.72, 1.5, 3.7] стало [3.7, 1.5, 1.72]
Не найден ключ александров владимир анатольевич в датасете
В датасете нарушена последовательность измерений для александрова валентина михайловна - в оригинале было [1.24, 1.19, 1.17] стало [1.17, 1.19, 1.24]
Не найден ключ алетдинов фавазит фаваритович в датасете
В датасете нарушена последовательность измерений для а

Могут быть небольшие неточности, но пока есть ощущение, что из 865 пациентов, которые я вытащил, в датасете присутствуют только 767, из которых у 135 нарушена последовательность взятия показателя МНО (то есть регрессия будет неверно считаться на них). Поэтому в датасете есть еще над чем поработать.

# Подгрузка данных доз и диагнозов

In [76]:
df = pd.read_excel('/content/BashCardio/Свод и чистка 8 больниц.xlsx', sheet_name = 1)
df

,Name,patient,gender,age,diagnos,sss,mg,AnalisDate,MNO,ИндексДиаг,ИндексПац,ИндексSSS
0,ЦАТ_SIBа.xlsx,Абдрахимов Гайфулла Рахимьянович,0,64.31759,ИБС,0,2.500,2022-11-28,2.46,3,484,1
1,ЦАТ_KMRTа.xlsx,Абдрахманов Гаряй Анварович,0,65.38261,ИБС ПС,ПК,7.500,2022-09-16,3.41,8,135,3
2,ЦАТ_KMRTа.xlsx,Абдрахманов Гаряй Анварович,0,65.38261,ПС,ПК,8.125,2022-11-02,1.89,2,135,3
3,ЦАТ_KMRTа.xlsx,Абдрахманов Гаряй Анварович,0,65.38261,ПС,ПК,8.125,2022-12-28,1.59,2,135,3
4,ЦАТ_KMRTа.xlsx,Абдрахманова Минигуль Габдуловна,1,69.91102,ПС,ПК,8.125,2022-03-23,2.87,2,136,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1912,ЦАТ_GKB5а.xlsx,Яхин Махмут Шарифуллович,0,76.47912,ФП,0,2.500,2022-12-21,2.43,1,73,1
1913,ЦАТ_GKB5а.xlsx,Яхин Махмут Шарифуллович,0,76.47912,ФП,0,2.500,2023-12-19,2.16,1,73,1
1914,ЦАТ_SIBа.xlsx,Яценко Людмила Афанасьевна,1,81.90007,ФП,0,2.500,2022-12-28,1.14,1,485,1
1915,ЦАТ_SIBа.xlsx,Яценко Людмила Афанасьевна,1,81.90007,ФП,0,2.500,2023-01-24,2.87,1,485,1


In [83]:
res.merge(df, how='left', on=['patient', 'AnalisDate'])

,patient,1,2,AnalisDate,4,Name,gender,age,diagnos,sss,mg,MNO,ИндексДиаг,ИндексПац,ИндексSSS
0,АБРАМОВ ТАТЬЯНА,0,76,2022-12-11,1.50,Чишминская ЦРБа.xlsx,1.0,75.23614,ФП,0,2.500,1.50,1.0,652.0,1.0
1,АБРАМОВ ТАТЬЯНА,0,76,2023-01-24,1.80,Чишминская ЦРБа.xlsx,1.0,75.23614,ФП,0,2.500,1.80,1.0,652.0,1.0
2,АГЕЕВ АНУВАР ГАБДУЛЛОВИЧ,1,76,2023-01-13,2.30,ЦАТ_GKB5а.xlsx,0.0,75.46886,ФП,0,4.375,2.30,1.0,88.0,1.0
3,АГЕЕВ АНУВАР ГАБДУЛЛОВИЧ,1,76,2022-12-26,1.89,ЦАТ_GKB5а.xlsx,0.0,75.46886,ФП,0,4.375,1.89,1.0,88.0,1.0
4,АГЕЕВ АНУВАР ГАБДУЛЛОВИЧ,1,76,2022-09-15,2.67,ЦАТ_GKB5а.xlsx,0.0,75.46886,ФП,0,4.375,2.67,1.0,88.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,мирзаянов Альфред Амирьянович,1,67,2023-01-13,3.00,ЦАТ_NEFTа.xlsx,0.0,66.75154,ПС,0,5.000,3.00,2.0,383.0,1.0
2571,мирзаянов Альфред Амирьянович,1,67,2022-12-09,4.36,ЦАТ_NEFTа.xlsx,0.0,66.75154,ПС,0,5.000,4.36,2.0,383.0,1.0
2572,тимофеев николай павлович,1,76,2022-12-21,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2573,тимофеев николай павлович,1,76,2023-01-09,2.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
